## Семинар 12

# Тема: Кросс-валидация

**Разбиение выборки** - это разделение имеющихся данных на несколько частей для проведения процессов обучения и оценки качества алгоритма машинного обучения на полностью независимых наборах данных.

### Способы разбиения выборки
1. **hold-out** разбиение (отложенная выборка, т.е. на две части - обучающую и тестовую)
2. **cross-validation** разбиение (на k наборов, имеющих обучающую и тестовую части)

    #### Виды разбиений для перекрёстной прорверки:
    1. **k-fold** (разбиение на k частей с примерно одинаковым количеством объектов в каждом, из которых получают k наборов обучающих и
    тестовых выборок, в которых одна часть выступает в роли тестовой выборки, а объединение остальных — в роли обучающей)
    2. **Stratified k-fold**(k-fold разбиение со стратификацией, т.е. все наборы содержат примерно такой же процент объектов каждого
    класса, что и исходные данные)
    2. **leave-one-out** (отложенный пример)
    и др.

**Кросс-валидация** (перекрёстная проверка) — это процедура для оценки обобщающей способности алгоритмов, состоящая из следующих этапов:

1. Фиксируется число блоков k. 

2. Выборка разбивается на k блоков определённым образом и получается k наборов данных, в которых один блок выступает в роли тестовой выборки, а объединение остальных блоков — в роли тренировочной. 

3. Осуществляется k итераций обучения модели на тренировочной выборке и подсчёт метрик на тестовой.

4. Итоговые метрики получаются усреднением k получившихся результатов.

Импортируем необходимые библиотеки:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.datasets import load_iris, load_wine, load_diabetes

from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut, cross_val_score, cross_validate

Загрузим встроенные данные об ирисах и выведем их описание:

In [3]:
iris = load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

Выделим признаки и целевую переменную:

In [4]:
X = iris.data
y = iris.target

Сделаем hold-out разбиение данных на обучающую и тестовую части:

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Обучим модель логистической регрессии на обучающих данных и найдём значение метрики accuracy на тестовых.

In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)
print('accuracy =', model.score(X_test, y_test))

accuracy = 1.0


Поскольку accuracy = 1, предполагаем, что модель всегда верно классифицирует данные. Проверим так ли это, проведя перекрёстную проверку.

Сделаем разбиение исходных данных на k-блоков с перемешиванием, где k = 3. Выведем индексы обучающих и тестовых частей каждого блока, а также их длины.

In [7]:
kf = KFold(n_splits = 3,shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Fold: {}, Длинна train: {}, Длинна test: {}".format(i, len(train_index), len(test_index)))
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold: 0, Длинна train: 100, Длинна test: 50
  Train: index=[  0   1   2   3   5   6   7   8  13  14  17  20  21  23  24  25  28  33
  34  35  37  38  39  40  41  43  44  46  47  48  49  50  52  53  54  57
  58  59  60  61  62  63  66  67  70  71  72  74  77  79  80  83  84  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 105 106
 107 111 112 113 114 115 116 117 119 120 121 122 123 124 125 126 129 130
 134 135 136 138 139 140 144 147 148 149]
  Test:  index=[  4   9  10  11  12  15  16  18  19  22  26  27  29  30  31  32  36  42
  45  51  55  56  64  65  68  69  73  75  76  78  81  82  85  86 104 108
 109 110 118 127 128 131 132 133 137 141 142 143 145 146]
Fold: 1, Длинна train: 100, Длинна test: 50
  Train: index=[  1   2   3   4   6   9  10  11  12  14  15  16  17  18  19  20  21  22
  26  27  29  30  31  32  36  37  38  41  42  45  46  48  50  51  52  54
  55  56  57  58  59  61  63  64  65  68  69  71  72  73  74  75  76  78
  79  81  82  85  86  87  88  90  91 

Сделаем k-блочную перекрёстную проверку (k = 3) модели логистической регрессии для метрики accuracy, используя библиотечную функцию cross_val_score.

In [8]:
cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = kf,                # тип разбиения (можно указать просто число фолдов cv = 3)
                             scoring = 'accuracy',   # метрика 'f1_macro','precision_macro', 'recall_macro'
                                                             # 'f1_micro','precision_micro', 'recall_micro'
                                                             # 'f1_weighted','precision_weighted', 'recall_weighted'
                             n_jobs=-1)              # используются все ядра CPU
cv_results

array([1.  , 0.92, 0.98])

Найдём среднее по кросс-валидации:

In [9]:
cv_results.mean()

np.float64(0.9666666666666667)

Делаем вывод, что модель классифицирует верно только 97% данных.

# Кросс-валидация в задаче классификации

### 1. Загрузите встроенный датасет o классификации вина из библиотеки sklearn.datasets. Выведите его описание. Обозначьте за _X_ двумерный numpy-массив признаков, а за _X_frame_ датафрейм признаков с соответствующими названиями колонок. Выведите размеры и первые пять строк, полученного датафрейма. Обозначьте за _y_ одномерный numpy-массив, а за _y_series_  - серию со значениями целевой переменной. Определите количество классов и количество объектов в каждом классе для целевой переменной. Убедитесь в том, что в данных нет пропущенных значений и они все числовые.
Указание: Для загрузки датасета [wine](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_wine.html) напишите wine = load_wine(). Для определения количества объектов в каждом классе используйте метод _.value_counts()_ библиотеки _pandas_.

In [9]:
wine = load_wine()


print(wine.DESCR)


X = wine.data
X_frame = pd.DataFrame(data=wine.data, columns=wine.feature_names)

print("Размеры датафрейма:", X_frame.shape)
print(X_frame.head())

y = wine.target
y_series = pd.Series(data=wine.target)

class_counts = y_series.value_counts()
print("Количество классов:", len(class_counts))
print("Количество объектов в каждом классе:\n", class_counts)

# Проверка отсутствия пропущенных значений и типа данных
print("Пропущенные значения в X_frame:", X_frame.isnull().sum().sum())
print("Типы данных в X_frame:\n", X_frame.dtypes)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

### 2. Сделайте hold-out (на две части) разбиение данных на обучающую и тестовую части в соотношении 4:1 тремя способами: без перемешивания и с перемешиванием двумя разными способами. Выведите количество объектов, попавшых в обучающую и в тестовую части. Выведите на экран индексы объектов обучающей и тестовой частей для каждого разбиения.
Указание: Используйте функцию train_test_split. Для разбиение без перемешивания, укажите параметр shuffle=False. Для разбиений с перемешиванием, зафиксируйте разные способы перемешивания, указав random_state=0 и  random_state=1. Для выведения индексов объектов, попавших в обучающую и тестовую выборки, передавайте в функцию датафрейм и серию (не numpy-массивы), а индексы выводите при помощи .index библиотеки pandas.

In [11]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_frame, y_series, test_size=0.2, shuffle=False)

# Разбиение с перемешиванием, метод 1
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_frame, y_series, test_size=0.2, shuffle=True, random_state=0)

# Разбиение с перемешиванием, метод 2
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_frame, y_series, test_size=0.2, shuffle=True, random_state=1)

# Вывод количества объектов и индексов: без перемешивания
print("Без перемешивания:")
print("Обучающая выборка:", len(X_train_1), "Тестовая выборка:", len(X_test_1))
print("Индексы обучающей:", X_train_1.index)
print("Индексы тестовой:", X_test_1.index)

# Вывод: с перемешиванием, метод 1
print("С перемешиванием (random_state=0):")
print("Обучающая выборка:", len(X_train_2), "Тестовая выборка:", len(X_test_2))
print("Индексы обучающей:", X_train_2.index)
print("Индексы тестовой:", X_test_2.index)



print("С перемешиванием (random_state=1):")
print("Обучающая выборка:", len(X_train_3), "Тестовая выборка:", len(X_test_3))
print("Индексы обучающей:", X_train_3.index)
print("Индексы тестовой:", X_test_3.index)

Без перемешивания:
Обучающая выборка: 142 Тестовая выборка: 36
Индексы обучающей: Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141],
      dtype='int64', length=142)
Индексы тестовой: Index([142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177],
      dtype='int64')
С перемешиванием (random_state=0):
Обучающая выборка: 142 Тестовая выборка: 36
Индексы обучающей: Index([161,  92,  94, 174,  24,  30,  93, 101, 113,  19,
       ...
        70,  87,  36,  21,   9, 103,  67, 117,  47, 172],
      dtype='int64', length=142)
Индексы тестовой: Index([ 54, 151,  63,  55, 123, 121,   7, 160, 106,  90, 141, 146,   5,  98,
       168,  80,  33,  18,  61,  51,  66,  37,   4, 104,  60, 111, 126,  86,
       112, 164,  26,  56, 129,  45,   8,  44],
      dtype='int64')
С перемешиванием (ra

### 3.Обучите модель логистической регрессии LogisticRegression(max_iter=5000) на трёх наборах обучающих данных, полученных в предыдущем задании. Сделайте предсказания на соответствующих трёх наборах тестовых данных и вычислите значения метрики accuracy для каждого из наборов. Сделайте вывод о том, влияет ли способ разбиения на значение метрики.

In [12]:
model = LogisticRegression(max_iter=5000)

# Обучение и предсказание для каждой части
# Без перемешивания
model.fit(X_train_1, y_train_1)
y_pred_1 = model.predict(X_test_1)
accuracy_1 = accuracy_score(y_test_1, y_pred_1)
print("Accuracy (без перемешивания):", accuracy_1)

# С перемешиванием (random_state=0)
model.fit(X_train_2, y_train_2)
y_pred_2 = model.predict(X_test_2)
accuracy_2 = accuracy_score(y_test_2, y_pred_2)
print("Accuracy (random_state=0):", accuracy_2)

# С перемешиванием (random_state=1)
model.fit(X_train_3, y_train_3)
y_pred_3 = model.predict(X_test_3)
accuracy_3 = accuracy_score(y_test_3, y_pred_3)
print("Accuracy (random_state=1):", accuracy_3)

# Вывод о влиянии способа разбиения
print("\nВывод: Способ разбиения влияет на значение метрики accuracy.")

Accuracy (без перемешивания): 1.0
Accuracy (random_state=0): 0.9722222222222222
Accuracy (random_state=1): 0.9444444444444444

Вывод: Способ разбиения влияет на значение метрики accuracy.


### 4. Сделайте разбиение исходных данных на k-блоков с перемешиванием, где k = 5. Выведите индексы обучающих и тестовых частей каждого блока.

Указание: Используйте класс [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) с параметрами n_splits = 5, shuffle=True, random_state=42.

In [23]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(kf.split(X_frame)):
    print(f"Fold {fold+1}")
    print("Индексы обучающей выборки:", train_index)
    print("Индексы тестовой выборки:", test_index)

Fold 1
Индексы обучающей выборки: [  0   1   2   3   4   5   6   7   8  10  11  13  14  17  20  21  22  23
  25  26  27  28  32  33  34  35  36  37  38  39  40  43  44  46  47  48
  49  50  51  52  53  54  56  57  58  59  61  62  63  64  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  83  84  85  86  87  88  89  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 110
 112 115 116 120 121 122 123 124 125 126 127 129 130 131 132 133 134 135
 136 137 138 139 142 143 144 146 147 148 149 151 152 153 154 155 156 157
 158 159 160 161 162 163 165 166 167 168 170 172 173 175 176 177]
Индексы тестовой выборки: [  9  12  15  16  18  19  24  29  30  31  41  42  45  55  60  65  66  67
  82  90 109 111 113 114 117 118 119 128 140 141 145 150 164 169 171 174]
Fold 2
Индексы обучающей выборки: [  0   1   3   5   7   8   9  10  12  13  14  15  16  17  18  19  20  21
  23  24  25  28  29  30  31  33  34  35  37  39  40  41  42  43  44  45
  46  47  48  49  50  52  53  54  55

### 5. Напишите свой код, осуществляющий k-блочную перекрёстную проверку (кросс-валидацию) с количеством блоков k = 5 модели логистической регрессии для метрики accuracy. Усредните полученный результат.

Указание: Используйте индексы блоков, полученных в предыдущем задании.

In [14]:
accuracies = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

mean_accuracy = np.mean(accuracies)
print("Среднее значение accuracy:", mean_accuracy)

Среднее значение accuracy: 0.9552380952380952


### 6. Сделайте k-блочную перекрёстную проверку с k = 5 модели логистической регрессии для метрики accuracy, используя библиотечную функцию cross_val_score. Убедитесь, что получится тот же результат, что и в предыдущем задании. Также выведите дисперсию по кросс-валидации.
Указание: Функция [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) позволяет осуществлять перекрёстную проверку для вычисления только одной метрики, которую нужно указать в параметре scoring.

In [15]:
scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
mean_score = np.mean(scores)
variance_score = np.var(scores)

print("Среднее значение accuracy с cross_val_score:", mean_score)
print("Дисперсия:", variance_score)

Среднее значение accuracy с cross_val_score: 0.9552380952380952
Дисперсия: 0.0011006802721088445


### 7. Сделайте k-блочную перекрёстную проверку с k = 5 модели логистической регрессии для двух метрик accuracy и F1-мера, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик на обучающих и тестовых данных. Усредните их значения.
Указание: Функция [cross_validate](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.cross_validate.html) позволяет осуществлять перекрёстную проверку нескольких метрик, которые нужно указать списком в параметре scoring.

In [16]:
scoring = ['accuracy', 'f1_weighted']

cv_results = cross_validate(model, X, y, cv=kf, scoring=scoring, return_train_score=True)

# Вывод результатов
print("Accuracy на обучающих данных:", cv_results['train_accuracy'])
print("Accuracy на тестовых данных:", cv_results['test_accuracy'])
print("F1 на обучающей:", cv_results['train_f1_weighted'])
print("F1 на тестовой:", cv_results['test_f1_weighted'])

mean_train_accuracy = np.mean(cv_results['train_accuracy'])
mean_test_accuracy = np.mean(cv_results['test_accuracy'])
mean_train_f1 = np.mean(cv_results['train_f1_weighted'])
mean_test_f1 = np.mean(cv_results['test_f1_weighted'])

print("\nСреднее значение accuracy на обучающих данных:", mean_train_accuracy)
print("Среднее значение accuracy на тестовых данных:", mean_test_accuracy)
print("Среднее значение F1 на обучающих данных:", mean_train_f1)
print("Среднее значение F1 на тестовых данных:", mean_test_f1)

Accuracy на обучающих данных: [0.99295775 0.99295775 1.         1.         0.98601399]
Accuracy на тестовых данных: [1.         0.91666667 0.91666667 0.97142857 0.97142857]
F1 на обучающей: [0.9929488  0.99294998 1.         1.         0.98602259]
F1 на тестовой: [1.         0.91586151 0.91613503 0.97153741 0.9711693 ]

Среднее значение accuracy на обучающих данных: 0.9943858957943466
Среднее значение accuracy на тестовых данных: 0.9552380952380952
Среднее значение F1 на обучающих данных: 0.9943842743204152
Среднее значение F1 на тестовых данных: 0.9549406531552872


### 8. Сделайте разбиение исходных данных на k-блоков со стратификацией, число блоков возьмите равным 5. Выведите индексы обучающих и тестовых частей каждого блока. Выведите для полученных блоков обучающих данных количество объектов каждого класса, сравните результаты с количеством объектов каждого класса для целевой переменной в целом.

Указание: Используйте класс [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) с параметрами n_splits = 5, shuffle=True, random_state=42. StratifiedKFold возвращает стратифицированные фолды, т.е. каждый блок содержит примерно такой же процент объектов каждого класса, что и полный набор.

In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(skf.split(X_frame, y_series)):
    print(f"Fold {fold+1}")
    print("Индексы обучающей выборки:", train_index)
    print("Индексы тестовой выборки:", test_index)
    
    y_train_fold = y_series.iloc[train_index]
    print("Количество объектов классов в обучающей выборке:\n", y_train_fold.value_counts())

Fold 1
Индексы обучающей выборки: [  2   3   4   5   6   7   8   9  10  12  13  15  18  19  20  21  22  23
  24  26  27  28  29  30  31  32  34  35  36  37  38  39  41  43  44  45
  46  47  49  50  51  52  54  55  56  57  58  59  60  61  62  63  64  66
  67  68  69  70  71  72  73  74  75  77  78  80  81  82  84  85  87  88
  91  93  94  95  96  97  98  99 100 101 102 103 104 105 106 108 110 111
 112 113 114 115 116 117 118 122 123 124 125 126 128 129 130 131 133 134
 135 136 137 138 139 141 142 143 144 147 148 149 150 151 152 153 154 155
 157 158 160 161 162 163 164 165 167 168 169 171 173 174 175 177]
Индексы тестовой выборки: [  0   1  11  14  16  17  25  33  40  42  48  53  65  76  79  83  86  89
  90  92 107 109 119 120 121 127 132 140 145 146 156 159 166 170 172 176]
Количество объектов классов в обучающей выборке:
 1    57
0    47
2    38
Name: count, dtype: int64
Fold 2
Индексы обучающей выборки: [  0   1   2   4   5   6   7   8  10  11  12  13  14  16  17  18  20  21
  22  23 

### 9. Сделайте k-блочную перекрёстную проверку (k = 5) со стратификацией модели логистической регрессии для двух метрик accuracy и F1-мера, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик на обучающих и тестовых данных. Усредните их значения.

Указание: Для того, чтобы выводились массивы со значениями метрик на обучающих данных необходимо указать значения параметра return_train_score=True.

In [18]:
cv_results_stratified = cross_validate(
    model, X, y, cv=skf, scoring=scoring, return_train_score=True
)

print("Accuracy на обучающих данных (стратифицированное):", cv_results_stratified['train_accuracy'])
print("Accuracy на тестовых данных (стратифицированное):", cv_results_stratified['test_accuracy'])
print("F1 на обучающей (стратифицированное):", cv_results_stratified['train_f1_weighted'])
print("F1 на тестовой (стратифицированное):", cv_results_stratified['test_f1_weighted'])

mean_stratified_train_accuracy = np.mean(cv_results_stratified['train_accuracy'])
mean_stratified_test_accuracy = np.mean(cv_results_stratified['test_accuracy'])
mean_stratified_train_f1 = np.mean(cv_results_stratified['train_f1_weighted'])
mean_stratified_test_f1 = np.mean(cv_results_stratified['test_f1_weighted'])

print("\nСреднее значение accuracy на обучающих данных (стратифицированное):", mean_stratified_train_accuracy)
print("Среднее значение accuracy на тестовых данных (стратифицированное):", mean_stratified_test_accuracy)
print("Среднее значение F1 на обучающих данных (стратифицированное):", mean_stratified_train_f1)
print("Среднее значение F1 на тестовых данных (стратифицированное):", mean_stratified_test_f1)

Accuracy на обучающих данных (стратифицированное): [0.99295775 1.         0.99295775 1.         0.99300699]
Accuracy на тестовых данных (стратифицированное): [0.94444444 0.97222222 0.97222222 0.94285714 1.        ]
F1 на обучающей (стратифицированное): [0.99294142 1.         0.9929505  1.         0.99300113]
F1 на тестовой (стратифицированное): [0.94450196 0.97226337 0.97197016 0.94285714 1.        ]

Среднее значение accuracy на обучающих данных (стратифицированное): 0.9957844971929479
Среднее значение accuracy на тестовых данных (стратифицированное): 0.9663492063492063
Среднее значение F1 на обучающих данных (стратифицированное): 0.9957786100451715
Среднее значение F1 на тестовых данных (стратифицированное): 0.9663185255975002


### 10. Сделайте leave-one-out разбиение данных. Выведите индексы обучающих и тестовых частей.

Указание: Используйте  класс [LeaveOneOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html).

In [19]:
loo = LeaveOneOut()
loo_indices = list(loo.split(X))

# Вывод индексов для каждого разбиения
for train_index, test_index in loo_indices:
    print("Индексы обучающей выборки:", train_index)
    print("Индексы тестовой выборки:", test_index)

Индексы обучающей выборки: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177]
Индексы тестовой выборки: [0]
Индексы обучающей выборки: [  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  4

### 11. Сделайте перекрёстную проверку модели логистической регрессии для метрик accuracy и F1-мера, изпользуя leave-one-out разбиение. Выведите полученные массивы со значениями метрик на тестовых данных. Усредните их значения.

In [20]:
accuracy_scores_loo = []
f1_scores_loo = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy_scores_loo.append(accuracy_score(y_test, y_pred))
    f1_scores_loo.append(f1_score(y_test, y_pred, average='weighted'))

mean_accuracy_loo = np.mean(accuracy_scores_loo)
mean_f1_loo = np.mean(f1_scores_loo)

print("Среднее значение accuracy (Leave-One-Out):", mean_accuracy_loo)
print("Среднее значение F1 (Leave-One-Out):", mean_f1_loo)

Среднее значение accuracy (Leave-One-Out): 0.9550561797752809
Среднее значение F1 (Leave-One-Out): 0.9550561797752809


### 12. Напишите свой код, осуществляющий k-блочную перекрёстную проверку (k = 5) модели логистической регрессии для метрик accuracy и F1_мера с предварительным масштабированием данных методом стандартной нормализации. Усредните полученные результаты. 

In [21]:
scaler = StandardScaler()
pipeline = make_pipeline(scaler, model)

accuracies_scaled = []
f1_scores_scaled = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    accuracies_scaled.append(acc)
    f1_scores_scaled.append(f1)

mean_accuracy_scaled = np.mean(accuracies_scaled)
mean_f1_scaled = np.mean(f1_scores_scaled)

print("Среднее значение accuracy (с масштабированием):", mean_accuracy_scaled)
print("Среднее значение F1 (с масштабированием):", mean_f1_scaled)

Среднее значение accuracy (с масштабированием): 0.9833333333333332
Среднее значение F1 (с масштабированием): 0.983341708891215


### 13. Сделайте k-блочную перекрёстную проверку (k = 5) модели логистической регрессии для двух метрик accuracy и F1-мера с предварительным масштабированием данных методом стандартной нормализации, используя библиотечную функцию cross_validate. Усредните значения метрик.

Указание: Используйте конвейер, передайте его в функцию cross_validate в качестве параметра model.

In [22]:
cv_results_scaled = cross_validate(
    pipeline, X, y, cv=skf, scoring=scoring, return_train_score=True
)

print("Accuracy на обучающих данных (масштабированные):", cv_results_scaled['train_accuracy'])
print("Accuracy на тестовых данных (масштабированные):", cv_results_scaled['test_accuracy'])
print("F1 на обучающей (масштабированные):", cv_results_scaled['train_f1_weighted'])
print("F1 на тестовой (масштабированные):", cv_results_scaled['test_f1_weighted'])

mean_scaled_train_accuracy = np.mean(cv_results_scaled['train_accuracy'])
mean_scaled_test_accuracy = np.mean(cv_results_scaled['test_accuracy'])
mean_scaled_train_f1 = np.mean(cv_results_scaled['train_f1_weighted'])
mean_scaled_test_f1 = np.mean(cv_results_scaled['test_f1_weighted'])

print("\nСреднее значение accuracy на обучающих данных (масштабированные):", mean_scaled_train_accuracy)
print("Среднее значение accuracy на тестовых данных (масштабированные):", mean_scaled_test_accuracy)
print("Среднее значение F1 на обучающих данных (масштабированные):", mean_scaled_train_f1)
print("Среднее значение F1 на тестовых данных (масштабированные):", mean_scaled_test_f1)

Accuracy на обучающих данных (масштабированные): [1. 1. 1. 1. 1.]
Accuracy на тестовых данных (масштабированные): [0.97222222 0.97222222 0.97222222 1.         1.        ]
F1 на обучающей (масштабированные): [1. 1. 1. 1. 1.]
F1 на тестовой (масштабированные): [0.97236919 0.97236919 0.97197016 1.         1.        ]

Среднее значение accuracy на обучающих данных (масштабированные): 1.0
Среднее значение accuracy на тестовых данных (масштабированные): 0.9833333333333332
Среднее значение F1 на обучающих данных (масштабированные): 1.0
Среднее значение F1 на тестовых данных (масштабированные): 0.983341708891215


# Кросс-валидация в задаче регрессии

### 1. Загрузите встроенный датасет o прогрессировании диабета из библиотеки sklearn.datasets. Выведите его описание. Обозначьте за _X_ двумерный numpy-массив признаков, а за _X_frame_ датафрейм признаков с соответствующими названиями колонок. Выведите размеры и первые пять строк, полученного датафрейма. Обозначьте за _y_ одномерный numpy-массив, а за _y_series_  - серию со значениями целевой переменной. Выведите количество уникальных значений целевой переменной. Убедитесь в том, что в данных нет пропущенных значений и они все числовые.
Указание: Для загрузки датасета [diabetes](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_wine.html) напишите diabetes = load_diabetes(). Для определения количества объектов в каждом классе используйте метод _.value_counts()_ библиотеки _pandas_.

### 2. Сделайте hold-out (на две части) разбиение данных на обучающую и тестовую части в соотношении 4:1, зафиксировав способ перемешивания данных, указав значение параметра random_state=5. Выведите количество объектов, попавших в обучающую и тестовую части. Обучите модель линейной регрессии на обучающих данных и сделайте предсказание на тестовых данных. Вычислите значения метрик коэффициент детерминации и RMSE. На основе полученных значений сделайте предположение о качестве модели.

### 3. Напишите свой код, осуществляющий k-блочную перекрёстную проверку с количеством блоков k = 10 модели линейной регрессии для метрик коэффициент детерминации и RMSE. Усредните полученные результаты. Сделайте окончательный вывод о возможности пименения модели линейной регрессии к этим данным.

### 4. Сделайте k-блочную перекрёстную проверку (k = 10) модели линейной регрессии для двух метрик коэффициент детерминации и ошибка RMSE, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик и усредните их. Сравните полученные результаты с предыдущим пунктом.

Указание: Укажите параметр scoring = ['r2', 'neg_root_mean_squared_error']

### 5. Вместо модели линейной регрессии, примените полиномиальную модель второй степени. Сделайте k-блочную перекрёстную проверку (k = 10) этой модели для метрик коэффициент детерминации и ошибки RMSE с предварительным масштабированием данных min-max методом, используя библиотечную функцию cross_validate. Усредните полученное значения метрик. Используйте конвейер make_pipeline. Построенная полиномиальная модель описывает данные лучше линейной модели или нет?